# Feasibility modeling



A more complicated example involves minimizing in two dimensions, where some parts of the parameter space are off-limits. Let's minimize Himmelblau's function, subject to the constraint that $x_1^2 + x_2^2 < 36$

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from bloptools import test_functions

x1 = x2 = np.linspace(-8, 8, 256)
X1, X2 = np.meshgrid(x1, x2)

F = test_functions.himmelblau(X1, X2)
F[X1 ** 2 + X2 ** 2 > 50] = np.nan

plt.pcolormesh(x1, x2, F, norm=mpl.colors.LogNorm(), shading="auto")
plt.colorbar()
plt.xlabel("x1")
plt.ylabel("x2")


In our digestion function, we return a `NaN` when we violate the constraint:

In [ ]:
def digestion(db, uid):

    products = db[uid].table()

    for index, entry in products.iterrows():

        if entry.x1 ** 2 + entry.x2 ** 2 < 50:
            products.loc[index, "himmelblau"] = test_functions.himmelblau(entry.x1, entry.x2)
        else:
            products.loc[index, "himmelblau"] = np.nan

    return products

and create the agent in the usual way:

In [ ]:
%run -i ../../../examples/prepare_bluesky.py # prepare the bluesky environment

agent = bloptools.bayesian.Agent(
                                active_dofs=dofs,
                                passive_dofs=[],
                                active_dof_bounds=bounds,
                                tasks=[task],
                                digestion=digestion,
                                db=db,
                                )

RE(agent.initialize(init_scheme='quasi-random', n_init=16))

agent.plot_tasks()


In addition to modeling the fitness of the task, the agent models the probability that an input will be feasible:

In [ ]:
agent.plot_feasibility()

The agent automatically tries to avoid infeasible points, but will end up naturally exploring the boundary of the constraint. Let's see where the agent is thinking of going:

In [ ]:
agent.plot_acquisition(strategy=["ei", "pi", "ucb"])

In [ ]:
RE(agent.learn(strategy='ei', n_iter=4))
agent.plot_tasks()

The agent will naturally explore the whole parameter space

In [ ]:
RE(agent.learn(strategy='ei', n_iter=16))
agent.plot_tasks()